In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Verificar las columnas disponibles
print("Columnas originales:", dataset.columns.tolist())

# Identificar la columna objetivo (puede ser 'expenses' o 'charges')
target_col = 'expenses' if 'expenses' in dataset.columns else 'charges'

# Separar características y etiquetas
labels = dataset.pop(target_col)

# Convertir variables categóricas (sex, smoker, region)
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')

# Dividir en entrenamiento (80%) y prueba (20%)
train_dataset = dataset.sample(frac=0.8, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

train_labels = labels[train_dataset.index]
test_labels = labels[test_dataset.index]

# Capa de normalización (versión actual de TensorFlow)
normalizer = layers.Normalization()
normalizer.adapt(np.array(train_dataset))

# Construir el modelo
model = keras.Sequential([
    normalizer,
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# Compilar el modelo
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='mae',
    metrics=['mae']
)

# Entrenar el modelo
history = model.fit(
    train_dataset,
    train_labels,
    epochs=150,
    batch_size=32,
    validation_split=0.2,
    verbose=0
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
